# Week 2 assignment:
# - Name: Justin Jeremiah Rangad
# - Roll Number: EE21B062

# Speed of computation

Python, being an interpreted language, tends to be slower than compiled languages like C or Fortran.  Some other languages like Java and Julia tend to use Just-in-Time compilation which can give speedups, but Python also has the problem of being dynamically typed, which eliminates the possibility of many optimizations.

The `timeit` library provides functions to estimate the time taken to run a piece of code.  It can automatically run the code multiple times to get better average results, and can be used to identify bottlenecks in your program.  However, it should be used with care as it is not a detailed function-call-level profiler.

It can either be `import`ed as a module where you can then explicitly called `timeit.timeit(func)` to estimate time for a function, or you can use the *magic syntax* in Python notebooks as shown below.  

import numpy as np
x = np.random.rand(10000,1)

In [3]:
import numpy as np
def sumarr(x):
    sum = 0
    for i in range(len(x)): 
    # for i in x:
        sum += x[i]
    return sum
print(x)
print(sumarr(x))
%timeit sumarr(x)

[[0.69637791]
 [0.2452843 ]
 [0.33516839]
 [0.31404717]
 [0.69928845]
 [0.95607886]
 [0.78205829]
 [0.21076512]
 [0.47358784]
 [0.46714348]]
[5.17979982]
15.7 µs ± 653 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [5]:
import numpy as np
def npsumarr(x):
    return np.sum(x)
print(npsumarr(x))
%timeit npsumarr(x)

5.179799822818478
4.97 µs ± 428 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


# Solving equations by Gaussian elimination

Once you have constructed two matrices A and B to represent the system of linear equations 
$$ Ax = b $$
you can then proceed to solve the equations using the process known as Gaussian elimination.

It is assumed you already know how the process works, but to refresh your memory, you could use the reference material at [LibreTexts](https://math.libretexts.org/Bookshelves/Algebra/Book%3A_Algebra_and_Trigonometry_(OpenStax)/11%3A_Systems_of_Equations_and_Inequalities/11.06%3A_Solving_Systems_with_Gaussian_Elimination).

Basically it involves making the A matrix *triangular* and ultimately into the shape of an identity matrix.

In [26]:
# Input matrices - the set of equations - 2 variables x1 and x2
A = [ [2,3], [1,-1] ]
B = [6,1/2]
print(A)
print(B)

[[2, 3], [1, -1]]
[6, 0.5]


In [27]:
# Normalize row 1
norm = A[0][0]
for i in range(len(A[0])): A[0][i] /= norm
B[0] = B[0]/norm
print(A)
print(B)

[[1.0, 1.5], [1, -1]]
[3.0, 0.5]


In [28]:
# Eliminate row 2 - A[1] - need to check and ensure div-by-zero etc doesnt happen
norm = A[1][0] / A[0][0]
for i in range(len(A[1])): A[1][i] = A[1][i] - A[0][i] * norm
B[1] = B[1] - B[0] * norm
print(A)
print(B)

[[1.0, 1.5], [0.0, -2.5]]
[3.0, -2.5]


In [29]:
# Normalize row 2 - B[1] will now contain the solution for x2
norm = A[1][1]
for i in range(len(A[1])): A[1][i] = A[1][i] / norm
B[1] = B[1] / norm
print(A)
print(B)

[[1.0, 1.5], [-0.0, 1.0]]
[3.0, 1.0]


In [30]:
# Sub back and solve for B[0] <-> x1
# This can be seen as eliminating A[0][1]
norm = A[0][1] / A[0][0]
# note that len(A) will give number of rows
for i in range(len(A)): 
    A[i][1] = A[i][1] - A[i][0] * norm
    B[i] = B[i] - A[i][0] * norm
print(A)
print(B)

[[1.0, 0.0], [-0.0, 1.0]]
[1.5, 1.0]


## Problems with Gaussian elimination

There are several obvious problems with the method outlined here.  These include:

- Performance - Python lists are not the most efficient way to store matrices
- Zeros: the simple example does not consider a scenario where one of the values on the diagonal may be 0.  Then some shuffling of rows is required.
- Numerical stability: there are several *normalization* steps involved, where it is quite possible for the values to blow up out of control if not managed properly.  Usually some kind of pivoting techniques are used to get around these issues.

In [31]:
import numpy as np
A1 = np.array( [ [2,3], [1,-1] ] )
B1 = np.array( [6, 1/2] )
np.linalg.solve(A1, B1)

array([1.5, 1. ])

# Library functions

*Numeric Python* or `numpy` is a library that allows Python code to directly call highly efficient implementations of several linear algebra routines (that have been written and optimized using C or Fortran and generally offer very high performance).

Although you can use the same `list` based approach to create matrices, it is better to declare them as the `array` data type:

- the numeric `type` (float, int etc.) can be specified for arrays
- memory allocation is done more efficiently by assuming they are not meant to be arbitrarily extended or changed

# SPICE basics

Our goal is to implement a SPICE simulator.  In order to do this, we first need to read in the circuit description from a text file.  To start with, we will only consider the basic elements of SPICE: Voltage sources, Current sources, and Resistors.  A typical SPICE netlist would look like this:

```spice
.circuit
R1 GND 1 1  
R2 1 2 1    
V1 GND 2 dc 2
.end
```

This is basically a *netlist* with 3 *nodes* - one of which is Ground (GND) which is assumed to be have a voltage of 0V.  We can write down Kirchhoff's current law (KCL) equations at each node, to account for current balance.  In addition, we will have some equations that specify the voltages between nodes having a direct voltage source, since there is no resistance there to provide an equation.

For the above example, the equations will be

$$
\begin{aligned}
\frac{V1-0}{R1} & + & \frac{V1-V2}{R2} & = & 0 \\
\frac{V2-V1}{R2} & + & I1 & = & 0 \\
V2 & - & 0 & = & 2
\end{aligned}
$$

which can be written in Matrix form as:

$$
\begin{bmatrix}
\frac{1}{R1}+\frac{1}{R2} & \frac{-1}{R2} & 0 \\
\frac{-1}{R2}   & \frac{1}{R2}  & 1 \\
0  & 1  & 0
\end{bmatrix}
\begin{bmatrix}
V1 \\
V2 \\
I1
\end{bmatrix}
=
\begin{bmatrix}
0 \\
0 \\
2
\end{bmatrix}
$$

At this point, you have reduced the solving of the SPICE equations to a known problem (linear equation solving) that you already know how to do.

## AC sources

The assumption above is that the system consists only of Voltage or Current sources and resistors.  What about capacitors, inductors, and AC sources?  These can be handled in exactly the same way as long as the circuit is operating at a single frequency.  We then replace the elements with their corresponding *impedance* values, which are frequency dependent complex numbers, but since there is only a single frequency they will still be constants.

## Problem scenarios

- Voltage source loops
- Current sources at a node
- Circuit defined with both DC and AC sources
- Syntax errors

# String and File manipulation

Given a SPICE netlist like the one above, you need to *read* it and construct an internal matrix as described above.  For string manipulation, there are a few helpful utility functions that we can see here.

In [2]:
circ = """.circuit
R1 GND 1 1  
R2 1 2 1    
V1 GND 2 dc 2
.end
""".splitlines()
print(circ)
for l in circ:
    if l[0] == 'R':
        print("Found a resistor")
    elif l[0] == 'V':
        print("Found a voltage source with value: ", float(l.split()[4]))

['.circuit', 'R1 GND 1 1  ', 'R2 1 2 1    ', 'V1 GND 2 dc 2', '.end']
Found a resistor
Found a resistor
Found a voltage source with value:  2.0


## Files

You can read from a file using the `readlines()` method of file objects.  One thing to keep in mind is how you open and close file objects.  In particular, it is strongly recommended to use the pattern `with open("filename") as f:` to ensure that the file is closed once you are done with reading it.  

# Assignment

The following are the problems you need to solve for this assignment.  You need to submit your code (either as standalone Python script or a Python notebook), a PDF document explaining your solution (either generated from the notebook or a 
separate LaTeX document), and any supporting files you may have (such as circuit netlists you used for testing your code).

- Write a function to find the factorial of N (N being an input) and find the time taken to compute it.  This will obviously depend on where you run the code and which approach you use to implement the factorial.  Explain your observations briefly.


There are two ways I approached this problem:
- Using iteration (Down to Up) approach
- Using Recursion (Up to Down) approach

Using iteration:
    We can find the factorial of a number simply by using a for loop and taking the product of the iterating variable i with the previous computed factorial i.e the factorial of i-1.
    
    The code below implements that:

In [34]:
def fact_1(n):
    a = 1                #a stores the result
    for i in range(2, n+1):
        a *= i
    return a
print(fact_1(50))
%timeit fact_1(50)

30414093201713378043612608166064768844377641568960512000000000000
2.35 µs ± 108 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


As we can see the code is pretty efficient and this approach is called the down to up approach
of dynamic programming.

Using Recursion:
    We can also use a recursive method to solve this problem. This method is however is slower as function calls are slower than for loop iterations. We basically imbed the function within its own definition to solve the problem. It is similar to finding the nth fibonnaci number.

In [35]:
import math
def fact_2(n):#Recursive approach
    if n == 1:
        return 1
    return (n*fact_2(n-1)) #Recalling the function
print(fact_2(50))
print(math.factorial(50))
%timeit fact_2(50)
%timeit math.factorial(50)

30414093201713378043612608166064768844377641568960512000000000000
30414093201713378043612608166064768844377641568960512000000000000
5.61 µs ± 162 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
371 ns ± 8.11 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


As we can see the Recursive function is a bit slower than iteration but it pales in comparison to the inbuilt optimized factorial function

As we can see this approach is a bit slower than the other approach.


- Write a linear equation solver that will take in matrices $A$ and $b$ as inputs, and return the vector $x$ that solves the equation $Ax=b$.  Your function should catch errors in the inputs and return suitable error messages for different possible problems.
  - Time your solver to solve a random $10\times 10$ system of equations.  Compare the time taken against the `numpy.linalg.solve` function for the same inputs.

## Gaussian Elimination
To solve this problem we must first understand how gaussian elimination works and how to immplement it algorithmically:
We need to convert all entries below the main diagonal to be zero. In order to do that we have to perform two operations, Normalisation and subtraction.

For a column:
1. To normalize we need to divide every entry of a row by the value of the entry of the row just below a diagonal element we then multiply the row with the value of diagonal element above.

2. Next we subtract all lower rows with the row of whom we took multiplied the value of the diagonal element

3. Incase there are any 0s in the diagonal we must switch rows so that we can compute the values for all variables

In [2]:
import numpy as np
def gaussian_elemination(A, b):
    n = len(b)
    if np.linalg.det(A)==0:
        return("The matrix is singular")
    #Gaussian Elimination:
    for k in range(n-1):
        if np.abs(A[k, k]) < np.abs(complex(1e-12, 0)):
            for l in range(k+1, n):       #Zero diagonal shifting
                if (A[l, k]) > complex(1e-12, 0):
                    A[[k, l]] = A[[l, k]]
                    b[[k, l]] = b[[l, k]]
                    break
        for i in range(k+1, n):
            if np.abs(A[i, k]) == np.abs(complex(0, 0)):
                continue
            a = A[k, k] / A[i, k] #normalizing factor
            for j in range(k, n): #Subtraction to form zeros
                A[i, j] = A[k, j]-A[i, j]*a
            b[i] = b[k]-b[i]*a
    # Back-substitution:
    sol = np.ones(n, dtype=np.complex_)
    sol[n-1] = b[n-1]/A[n-1, n-1]
    for i in range(n-2, -1, -1):
        s = 0
        for j in range(n-1, i, -1):
            s += A[i, j]*sol[j]
        sol[i] = (b[i]-s)/A[i, i]
    return(sol)
A=np.random.randint(0,100,size = (10,10))   #Random 10x10 matrix generator 
b=np.random.randint(0,100,size=(10))       
print(gaussian_elemination(A,b))
print(np.linalg.solve(A,b))
%timeit gaussian_elemination(A,b)
%timeit np.linalg.solve(A,b)

[ 1.15605528+0.j  0.80059368-0.j -0.23824074-0.j  0.36833333-0.j
 -0.1875    -0.j -0.38333333-0.j -0.41666667+0.j  0.70833333-0.j
  0.75      -0.j -0.5       +0.j]
[ 1.15605528  0.80059368 -0.23824074  0.36833333 -0.1875     -0.38333333
 -0.41666667  0.70833333  0.75       -0.5       ]
127 µs ± 8.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
19.2 µs ± 511 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


As we can see the numpy.linalg.solve function is faster than our code when tested. This is because numpy uses highly optimized C or Fortran code which is why it is much much faster. The above code goes through the gaussian elimination algorithm and returns a result array which contains the values of the variables.

- Given a circuit netlist in the form described above, read it in from a file, construct the appropriate matrices, and use the solver you have written above to obtain the voltages and currents in the circuit.  If you find AC circuits hard to handle, first do this for pure DC circuits, but you should be able to handle both voltage and current sources.


## Netlist to Matrix conversion:
We can implement the above in various ways the method ive used is known as modified nodal analysis or MNA in short. In MNA every component has a stamp on the matrix, For example a resistor between two nodes 1 and 2 will have a stamp of (1/R) for matrix entries (1,1) and (2,2) and (-1/R) for entries (1,2) and (2,1). This is equivalent to writing the nodal equations for n1 and n2 and using them to form the matrix, we are just using the pattern each component forms in the matrix instead so that our work is easier. Similarly other components have different stamp values and hence we can form our matrices A and b to solve the equation Ax=b. Inorder to do that we pass the matrices A and b as argument to our gaussian elimination solver which we defined earlier. The answer will give us the nodal voltages and current through any independent voltage source. 

In [2]:
import numpy as np
def nc(s):
    for i in s:
        if i.isdigit():
            return(i)
    return(s)
        
def parser_netlist(filename):
    with open(filename,'r') as f:
        ptr1=""
        ptr2=""
        circ=""
        s=0
        dc=0
        #To make sure no extra things are being read besides the netlist
        while ".circuit" not in ptr1:
            ptr1=f.readline()
            if ".circuit" in ptr1:
                while ".end" not in ptr2 :
                    ptr2=f.readline()
                    circ+=ptr2
                ptr2=f.readline()
                if ".ac" in ptr2:
                    circ+=ptr2
    #To find the frequenct of the circuit
    net=list(circ.splitlines())
    ac = net[len(net)-1].split()
    if  ".ac" in ac :
        s+=1
        omega=2*(np.pi)*float(ac[2])
    else:
        omega=0
    ac=net[len(net)-2].split()
    if ".ac" in ac:
        s+=1
        if s==2:
            return(1)
    count={}
    dim=0
    v_count=0
    n_count=0
    for i in net:
        l=i.split()
        if ".ac" in l:
            break
        if len(l)<4:
            continue
        if l[0][0]=="V":
            dim+=1
            v_count+=1
        if nc(l[1]).isdigit():
            if nc(l[1]) not in count:
                count[nc(l[1])]=1
                dim+=1
                n_count+=1
        if nc(l[2]).isdigit():
            if nc(l[2]) not in count:
                count[nc(l[2])]=1
                n_count+=1
                dim+=1
    G=np.zeros((dim,dim),dtype=np.complex_)
    b=np.zeros(dim,dtype=np.complex_)
    v=dim-1
    for l in net:
        l=l.split()
        if ".end" in l:
            break
        l[1]=nc(l[1])
        l[2]=nc(l[2])
      
        #parsing for Resistors
        if l[0][0]=="R":
            y=1/float(l[3])
            #case 1: Resistor between two nodes
            if "GND" not in l:
                k=int(l[1])-1
                j=int(l[2])-1
                G[k,k]=G[k,k]+y
                G[k,j]=G[k,j]-y
                G[j,j]=G[j,j]+y
                G[j,k]=G[j,k]-y
            #case 2: Resistor between node and GND
            else:
                if l[1][0]!="G":
                    k=int(l[1])-1
                    G[k,k]=G[k,k]+y
                else:
                    k=int(l[2])-1
                    G[k,k]=G[k,k]+y
        #parsing for Voltage sources
        if l[0][0]=="V":
            if "dc" in l:
                dc+=1
                if s>=2:
                    return(1)
                if s and dc >=1:
                    return(2)
            #if voltage source is between a node and a node:
            if "GND" not in l:
                k=int(l[1])-1
                j=int(l[2])-1
                G[k,v]=G[k,v]+1
                G[v,k]=G[v,k]+1
                G[j,v]=G[j,v]-1
                G[v,j]=G[v,j]-1
            # if voltage source is between node and GND:
            else:
                if "GND"==l[1]:
                    j=int(l[2])-1
                    G[j,v]=G[j,v]-1
                    G[v,j]=G[v,j]-1
                else:
                    k=int(l[1])-1
                    G[k,v]=G[k,v]+1
                    G[v,k]=G[v,k]+1
            # altering b array
            if l[3] == "ac":
                        phase = float(l[5])
                        volt = complex(float(l[4])*np.cos(phase), float(l[4])*np.sin(phase))
            else:
                        volt=float(l[4])

            b[v]=b[v]+volt
        #parsing for Current sources
        if l[0][0]=="I":
            if "GND" not in l:
                k=int(l[1])-1
                j=int(l[2])-1
                if l[3] == "ac":
                        phase = float(l[5])
                        c = complex(float(l[4])*np.cos(phase),float(l[4])*np.sin(phase))
                else:
                        c=float(l[4])
                b[k]=b[k]+c
                b[j]=b[j]-c
            else:
                if "GND"==l[1]:
                    j=int(l[2])-1
                    if l[3]=="ac":
                        phase=float(l[5])
                        c = complex(float(l[4])*np.cos(phase),
                                    float(l[4])*np.sin(phase))
                    else:
                        c=float(l[4])
                    b[j]=b[j]-c
                else:
                    k=int(l[1])-1
                    if l[3] == "ac":
                        phase = float(l[5])
                        c = complex(float(l[4])*np.cos(phase),
                                    float(l[4])*np.sin(phase))
                    else:
                        c = float(l[4])
                    b[k]=b[k]+c
        #parsing for inductors:
        if l[0][0]=="L":
            imp=1/complex(0,float(l[3])*omega)
            #inductor between two nodes
            if "GND" not in l:
                k = int(l[1])-1
                j = int(l[2])-1
                G[k, k] = G[k, k]+imp
                G[k, j] = G[k, j]-imp
                G[j, j] = G[j, j]+imp
                G[j, k] = G[j, k]-imp
            # case 2: inductor between node and GND
            else:
                if l[1][0]!="G":
                    k=int(l[1])-1
                    G[k,k]=G[k,k]+imp
                else:
                    k=int(l[2])-1
                    G[k,k]=G[k,k]+imp
        #parsing for capacitors:
        if l[0][0]=="C":
            imp =complex(0, float(l[3])*omega)
            # capacitor between two nodes
            if "GND" not in l:
                k = int(l[1])-1
                j = int(l[2])-1
                G[k, k] = G[k, k]+imp
                G[k, j] = G[k, j]-imp
                G[j, j] = G[j, j]+imp
                G[j, k] = G[j, k]-imp
            # case 2: capacitor between node and GND
            else:
                if l[1][0] != "G":
                    k = int(l[1])-1
                    G[k, k] = G[k, k]+imp
                else:
                    k = int(l[2])-1
                    G[k, k] = G[k, k]+imp                
    return [G,b,n_count,v_count]

## Solving Netlists:
Since the parser_netlist function has a file reading capability inbuilt we only need to specify the name of the circuit we wish to simulate for the 7 circuits given please enter your option below in the form cktn (for example: ckt1 or ckt6). The below code handles that request and adds the relevant paths so that the file module can read the file and the function can form the matrix.

In [ ]:
s=input("Please enter what ckt you want to solve (Must be in the form cktn):")
s+=".txt" 
l=parser_netlist(s)
if l==1:
    print("ERROR:Two frequencies in the circuit")
elif l==2:
    print("ERROR:DC and AC source")
else:
    sol=gaussian_elemination(l[0],l[1])
    for i in range(l[2]+l[3]):
        if i<l[2]:
            print(f"The voltage at n{i+1} is: {sol[i]:>54}")       #string formatting
        else:
            print(f"The current across V{i+1} is: {sol[i]:>50}")